Generate Bifuracation Diagram

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
from matplotlib.pyplot import figure
import latex

from scipy import interpolate

In [ ]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

plt.rcParams['text.usetex'] = True
plt.rcParams.update({
     "font.family": "serif",   # specify font family here
     "font.serif": ["Nimbus Roman"],  # specify font here
     "font.size":11})

In [ ]:
def Bifurcation(Param):
    %run ./Data_Generator/utils.ipynb
    
    Alpha = Param["Alpha"]
    InitialSolve = Param["InitialSolve"]
    Ndt = Param["Ndt"]
    
    r=[]
    s=[]
    for alpha in Alpha:
        Path_U_IS = 'Data_Generator/' + f'u_original_{alpha}_{InitialSolve}.npy'

        U_initial_solve = np.load(Path_U_IS, allow_pickle=True)[::20,-1]

        Path_FNO = 'Data_Generator/' + f'FNO_shockarray_{alpha}_{Ndt}.npy'

        FNO = np.load(Path_FNO, allow_pickle=True)[:,-1]

        FNO_unfine = np.append(U_initial_solve,FNO)

        time_uni = np.linspace(0,10000,FNO_unfine.shape[-1])

        time_uni_fine = np.linspace(0,10000,50*FNO_unfine.shape[-1])

        tck = interpolate.splrep(time_uni, FNO_unfine, s = 0)

        FNO_fine = interpolate.splev(time_uni_fine, tck, der=0)

        data_FNO = np.array([time_uni_fine,FNO_fine]).T

        maxima = find_maxima(storage=None, t_min=8500, t_max=10000,
                    plot=False, ret_idx=False, main=False, tus=data_FNO)
        
        for i in range(maxima.shape[0]):
            r.append(alpha)
            s.append(maxima[i,1])
            
    fig = figure(figsize=(7, 7), dpi=90)
    plt.plot(r,s,'k.')
    plt.xlabel(r'$\alpha$',fontsize = 18)
    plt.ylabel(r'$u_s$',fontsize = 18)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.ylim([1,1.4])
    plt.xlim([4.4,5.4])
    plt.title('Bifurcation Diagram FNO', fontsize = 18)
    plt.grid(True)
    plt.savefig("Figures/" + 'Bifurcation_Diagram.jpg')
    return r, s, data_FNO